## Gold Pipeline– Curated, Business‑Ready Data


#### Load data

In [0]:

df_silver= spark.read.table("workspace.default.df_silver_events")
display(df_silver.limit(5))

date,event_year,user_id,product_id,event_type,category_code,category,product_name,brand,price
2019-10-01,2019,518497482,11000101,view,appliances.kitchen.toster,appliances,kitchen,philips,28
2019-10-01,2019,521245875,39800121,view,apparel.shirt,apparel,shirt,dion,29
2019-10-01,2019,553019427,1005105,view,electronics.smartphone,electronics,smartphone,apple,1415
2019-10-01,2019,545053960,2702480,view,appliances.kitchen.refrigerators,appliances,kitchen,midea,124
2019-10-01,2019,545921034,8500124,view,others,others,Unspecified,Unspecified,214


## Create Dimension Tables
A dedicated schema named *ecommerce* was created to store all Gold‑layer tables, ensuring a clear, organized structure for curated dimensional, fact, and KPI datasets.


In [0]:
%sql
---CREATE CATALOG IF NOT EXISTS Gold;
---CREATE SCHEMA IF NOT EXISTS Gold.ecommerce;

#### Product Dimension

In [0]:
dim_product = (
    df_silver
    .select(
        "product_id",
        "product_name",
        "brand",
        "category",
        "category_code"
    )
    .dropDuplicates(["product_id"])
)

dim_product.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.dim_product")


#### Date Dimension

In [0]:
from pyspark.sql.functions import col, dayofweek, month, year, weekofyear

dim_date = (
    df_silver
    .select("date")
    .dropDuplicates()
    .withColumn("event_year", year("date"))
    .withColumn("event_month", month("date"))
    .withColumn("event_week", weekofyear("date"))
    .withColumn("day_of_week", dayofweek("date"))
)

dim_date.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.dim_date")


#### User Dimension

In [0]:
dim_user = (
    df_silver
    .select("user_id")
    .dropDuplicates()
)

dim_user.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.dim_user")


#### Create Fact Table (Event-Level)

In [0]:
from pyspark.sql.functions import col

fact_events = (
    df_silver
    .select(
        "date",
        "user_id",
        "product_id",
        "event_type",
        "price",
        "category",
        "brand"
    )
)

fact_events.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.fact_events")


# Create Aggregated Gold Tables (KPIs)

#### Daily Category Performance

In [0]:
from pyspark.sql.functions import count, avg, countDistinct

daily_category_perf = (
    fact_events
    .groupBy("date", "category")
    .agg(
        count("*").alias("total_events"),
        countDistinct("user_id").alias("unique_users"),
        avg("price").alias("avg_price")
    )
)

daily_category_perf.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.daily_category_performance")


#### Brand Performance

In [0]:
brand_perf = (
    fact_events
    .groupBy("brand")
    .agg(
        count("*").alias("total_events"),
        avg("price").alias("avg_price")
    )
)

brand_perf.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.brand_performance")


#### Product Popularity

In [0]:
product_popularity = (
    df_silver
    .groupBy("product_name")
    .agg(
        count("*").alias("total_views"),
        countDistinct("user_id").alias("unique_users")
    )
)

product_popularity.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.product_popularity")


In [0]:
from pyspark.sql.functions import count, sum, when, col

product_popularity = (
    df_silver
    .groupBy("product_id")
    .agg(
        count("*").alias("views"),
        sum(when(col("event_type") == "cart", 1).otherwise(0)).alias("carts"),
        sum(when(col("event_type") == "purchase", 1).otherwise(0)).alias("purchases")
    )
)

product_popularity.write.format("delta").mode("overwrite").saveAsTable("gold.ecommerce.product_popularity_2")


#### Category Price Distribution

In [0]:
from pyspark.sql.functions import min, max

category_price_dist = (
    fact_events
    .groupBy("category")
    .agg(
        min("price").alias("min_price"),
        max("price").alias("max_price"),
        avg("price").alias("avg_price")
    )
)

category_price_dist.write.format("delta").mode("overwrite").saveAsTable("Gold.ecommerce.category_price_distribution")


#### KPI category daily

In [0]:
%sql
CREATE OR REPLACE TABLE gold.ecommerce.kpi_category_daily AS
SELECT
  category,
  COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
  COUNT(CASE WHEN event_type = 'cart' THEN 1 END) AS carts,
  COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases
FROM workspace.default.df_silver_events
GROUP BY category;


num_affected_rows,num_inserted_rows


### In the Gold layer, I transformed the cleansed Silver data into curated, business‑ready tables by modeling dimensions, building a granular fact table, and generating aggregated KPI datasets. This final layer delivers structured, analytics‑optimized outputs that enable clear insights, efficient SQL exploration, and reliable downstream reporting